In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
class InductiveLPDataset(DisjointInductivePathDataset):
    """An inductive link prediction dataset for the ILPC 2022 Challenge."""

    
    
    
    def __init__(self , **kwargs):
        """Initialize the inductive link prediction dataset.

        :param size: "small" or "large"
        :param kwargs: keyword arguments to forward to the base dataset class, cf. DisjointInductivePathDataset
        """
        DATA_TYPE = "_fully_inductive.tsv"
        TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
        TEST_PATH = "MSCallGraph_test" + DATA_TYPE
        VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE
        INFERENCE_PATH = "MSCallGraph_inference" + DATA_TYPE


        super().__init__(
            transductive_training_path=os.getcwd()+"/"+TRAIN_PATH,
            inductive_inference_path=os.getcwd()+"/"+INFERENCE_PATH,
            inductive_validation_path=os.getcwd()+"/"+VALIDATE_PATH,
            inductive_testing_path=os.getcwd()+"/"+TEST_PATH,
            create_inverse_triples=True,
            eager=True,
            **kwargs
        )


In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
dataset = InductiveLPDataset()

In [5]:
model_name = 'nodepiece_inductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
num_tokens = 20
embedding_dim = 200

In [8]:
model = InductiveNodePiece(
        triples_factory=dataset.transductive_training,
        inference_factory=dataset.inductive_inference,
        random_seed = seed,
        loss = loss,
        num_tokens = num_tokens,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/9.06k [00:00<?, ?it/s]

No symbolic computation of output shape.


sampling:   0%|          | 0.00/3.79k [00:00<?, ?it/s]

No symbolic computation of output shape.
No cuda devices were available. The model runs on CPU


Number of parameters: 2600
Space occupied: 10400 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_inductive created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 1
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        mode=TRAINING,
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        mode=VALIDATION,
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.inductive_inference,
    evaluation_triples_factory = dataset.inductive_validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.transductive_training,
        model=model,
        mode=TRAINING,  # must be specified for the inductive setup
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.transductive_training,
        num_epochs=num_epochs,
#         callbacks="evaluation",
#         callback_kwargs=dict(
#             evaluator=valid_evaluator,
#             evaluation_triples=dataset.inductive_validation.mapped_triples,
#             prefix="validation",
#             frequency=1,
#             additional_filter_triples=dataset.inductive_inference.mapped_triples,
#         ),
#         stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cpu:   0%|          | 0/1 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/184 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 2.726568292340507


In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
          0
0  2.726568


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.transductive_training.mapped_triples,
        additional_filter_triples=[
        dataset.transductive_training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cpu:   0%|          | 0.00/23.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.003809   0.003223     0.002911
hits_at_1                     0.000681   0.000681     0.000681
hits_at_3                     0.002726   0.001107     0.001107
hits_at_5                     0.004302   0.002683     0.002087
hits_at_10                    0.006900   0.005452     0.005281
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.010055   0.008132     0.007309
hits_at_1                     0.002257   0.001491     0.001491
hits_at_3                     0.004685   0.003280     0.003280
hits_at_5                     0.005452   0.005196     0.005196
hits_at_10                    0.016398   0.008689     0.008348
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.006932   0.005678     0.005110
hits_at_1                     0.001469   0.001086     0.001086
hits_at_3                     0.003705  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_inference.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cpu:   0%|          | 0.00/3.13k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012769   0.012145     0.011759
hits_at_1                     0.006396   0.006396     0.006396
hits_at_3                     0.011513   0.011513     0.011513
hits_at_5                     0.013431   0.013431     0.013431
hits_at_10                    0.016629   0.016310     0.015670
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.033718   0.031339     0.030103
hits_at_1                     0.017589   0.016310     0.016310
hits_at_3                     0.023985   0.020787     0.020467
hits_at_5                     0.031340   0.029421     0.028782
hits_at_10                    0.063319   0.052127     0.049248
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.023243   0.021742     0.020931
hits_at_1                     0.011992   0.011353     0.011353
hits_at_3                     0.017749  

In [25]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cpu:   0%|          | 0.00/3.13k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012684   0.012041     0.011641
hits_at_1                     0.006396   0.006396     0.006396
hits_at_3                     0.011193   0.011193     0.011193
hits_at_5                     0.013112   0.013112     0.013112
hits_at_10                    0.016629   0.016310     0.015670
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.032728   0.030165     0.028776
hits_at_1                     0.016310   0.014391     0.014391
hits_at_3                     0.023665   0.020467     0.020147
hits_at_5                     0.031020   0.029101     0.028782
hits_at_10                    0.057243   0.051167     0.047969
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.022706   0.021103     0.020209
hits_at_1                     0.011353   0.010393     0.010393
hits_at_3                     0.017429  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_inference.mapped_triples,
            dataset.inductive_validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cpu:   0%|          | 0.00/3.98k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.013561   0.012865     0.012434
hits_at_1                     0.006533   0.006533     0.006533
hits_at_3                     0.013065   0.013065     0.013065
hits_at_5                     0.014573   0.014322     0.014322
hits_at_10                    0.017588   0.017337     0.017085
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.028673   0.026226     0.024982
hits_at_1                     0.013065   0.012060     0.012060
hits_at_3                     0.019849   0.016583     0.016583
hits_at_5                     0.024623   0.022362     0.021608
hits_at_10                    0.055528   0.046482     0.042462
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.021117   0.019546     0.018708
hits_at_1                     0.009799   0.009296     0.009296
hits_at_3                     0.016457  

In [23]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cpu:   0%|          | 0.00/3.98k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.013561   0.012865     0.012434
hits_at_1                     0.006533   0.006533     0.006533
hits_at_3                     0.013065   0.013065     0.013065
hits_at_5                     0.014573   0.014322     0.014322
hits_at_10                    0.017588   0.017337     0.017085
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.028673   0.026226     0.024982
hits_at_1                     0.013065   0.012060     0.012060
hits_at_3                     0.019849   0.016583     0.016583
hits_at_5                     0.024623   0.022362     0.021608
hits_at_10                    0.055528   0.046482     0.042462
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.021117   0.019546     0.018708
hits_at_1                     0.009799   0.009296     0.009296
hits_at_3                     0.016457  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


FileNotFoundError: [WinError 2] Impossibile trovare il file specificato

In [ ]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [ ]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

In [ ]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)